# Map Reduce Oefening - Schaken

**Deadline:**

In deze oefening gaan we werken met een dataset dat informatie bevat over een groot aantal schaakspelletjes.
Deze dataset kan [hier](https://www.kaggle.com/datasnaek/chess) gevonden worden.
Om te beginnen, download deze file en upload hem naar je distributed file systeem onder het path: **Oefingen/Mapreduce**.
Voorzie ook code die deze folder reset naar een lege folder om geen naam-conflicten te hebben.
Om je code te testen, print de eerste 1000 bytes uit.

In [ ]:
# download dataset
import opendatasets as od

od.download("https://www.kaggle.com/datasets/datasnaek/chess")

In [1]:
#upload csv file to hdfs
from hdfs import InsecureClient

map = 'Oef_MapReduce'

client = InsecureClient('http://localhost:9870', user='bigdata')

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        if not f.endswith('games.csv'):
            client.delete(map + '/' + f, recursive=True)
            
if client.status(map + '/games.csv', strict=False) is None:
    client.upload(map, './chess/games.csv')

with client.read(map + '/games.csv', length=1000) as reader:
    content = reader.read()
    print(content.decode('utf-8'))

id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
TZJHLljE,FALSE,1.50421E+12,1.50421E+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4,D10,Slav Defense: Exchange Variation,5
l1NXvwaE,TRUE,1.50413E+12,1.50413E+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4 Bb4+,B00,Nimzowitsch Defense: Kennedy Variation,4
mIICvQHh,TRUE,1.50413E+12,1.50413E+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6 bxc6 Ra6 Nc4 a4 c3 a3 Nxa3 Rxa3 Rxa3 c4 dxc4 d5 cxd5 Qxd5 exd5 Be6 Ra8+ Ke7 Bc5+ Kf6 Bxf8 Kg6 Bxg7 Kxg7 dxe6 Kh6 exf7 Nf6 Rxh8 Nh5 Bxh5 Kg5 Rxh7 Kf5 Qf3+ Ke6 Bg4+ Kd6 Rh6+ Kc5 Qe3+ Kb5 c4+ Kb4 Qc3+ Ka4 Bd1#,C20,King's Pawn Game: Leonardis Variation,3
kWKvrqYL,TRUE,1.50411E+12,1.50411E+12,61,mate,white,20+0,daniamurashov,1439,adivanov


# Map Reduce applicaties

Nu dat de dataset geupload is kunnen we map-reduce applicaties schrijven om deze dataset te verwerken. Schrijf hieronder de nodige mapreduce applicaties om de elke oefening uit te voeren.
Deze oefeningen bestaan steeds uit twee cellen:
* Een eerste cel met de nodige python code voor de map-reduce applicatie. Sla deze code op in een file met het correcte oefeningennummer (bvb: oefening_1.py) door gebruik te maken van de "%%file" tag in de notebookcellen.
* Een tweede cel die de map-reduce applicatie uitvoert op de cluster en het correcte bestand uitleest. Commando's uitvoeren naar de commandline/terminal gebeurt door een uitroepingsteken vooraan te plaatsen (!pydoop submit ...)

**Maak gebruik van de mrjob package** in python en niet van de java code of andere aangehaalde libraries.

## Dimensies

De eerste gevraagde map-reduce applicatie telt het aantal rijen en het aantal kolommen.
Schrijf deze nu hieronder en beantwoord de vragen na de code:

In [4]:
%%file oefening_1.py
from mrjob.job import MRJob

class Vraag1(MRJob):

    def mapper(self, _, line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return

        yield("row", 1)
        yield("columns", line.count(",") + 1)

    def reducer(self, key, counts):
        # dit is per key, je kan niet van de ene key aan data van een andere key

        if key == "row":
            yield("aantal rijen", sum(counts))
        elif key == "columns":
            yield("aantal kolommen", max(counts))
        

# de main functie komt hieronder
if __name__ == "__main__":
    Vraag1.run()

Overwriting oefening_1.py


In [5]:
# commando voor uitvoeren van de applicatie
!python3 oefening_1.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/oefening_1.root.20240319.074319.171897
uploading working dir files to hdfs:///user/root/tmp/mrjob/oefening_1.root.20240319.074319.171897/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/oefening_1.root.20240319.074319.171897/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar8129976839371676184/] [] /tmp/streamjob7734138006844556987.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032
  Connecting to Application History server at historyserver/172.20.0.9:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032

**Vragen:**

- Hoeveel rijen heb je gevonden in de dataset?
- Hoeveel spelletjes zitten er echt in de dataset (kijk hiervoor naar de link van de dataset)?
- Zijn deze waarden gelijk? Indien nee, wat is de oorzaak van het verschil? Hoe zou je dit kunnen oplossen?

**Antwoord:**

Schrijf hier je antwoord...

Indien je waarden niet overeenkomen, maak een kopie van de vorige applicatie en pas het aan zodat het aantal spelletjes in de dataset correct geteld wordt.

In [ ]:
# aangepast map-reduce applicatie om het correcte aantal te bekomen.
%%file oefening1_gelijk.py

In [ ]:
# commando voor uitvoeren van de applicatie

## Verschil in rating

Wat is het gemiddelde verschil in de rating tussen de beide spelers? Hoe vaak wint de sterkste speler (geef dit terug als percentage van het aantal geregistreerde spelletjes)?

In [9]:
%%file oefening_2.py
from mrjob.job import MRJob
import csv
from io import StringIO

col_winner = 6
col_white_rating = 9
col_black_rating = 11

class Vraag2(MRJob):

    def mapper(self, _, line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return
        
        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file))  

        speler1 = float(cols[col_white_rating])
        speler2 = float(cols[col_black_rating])
        winner = cols[col_winner]

        yield("verschil", abs(speler1-speler2))

        if winner=="white" and speler1 >= speler2:
            yield("sterkste", 1)
        elif winner=="black" and speler2 >= speler1:
            yield("sterkste", 1)
        else:
            yield("sterkste", 0)

    def reducer(self, key, counts):
        # dit is per key maar beide keys kunnen op dezelfde manier verwerkt worden

        counts = list(counts)

        yield(key, sum(counts)/ len(counts))
        

# de main functie komt hieronder
if __name__ == "__main__":
    Vraag2.run()

Overwriting oefening_2.py


In [10]:
# commando voor uitvoeren van de applicatie
!python3 oefening_2.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/oefening_2.root.20240319.075302.351123
uploading working dir files to hdfs:///user/root/tmp/mrjob/oefening_2.root.20240319.075302.351123/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/oefening_2.root.20240319.075302.351123/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar6283131116140561786/] [] /tmp/streamjob2554835464661704698.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032
  Connecting to Application History server at historyserver/172.20.0.9:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032

## Uitkomsten per kleur

Schrijf nu een map-reduce applicatie dat telt welke uitkomst (status) het vaakst leidt tot een winnaar van de witte of zwarte kleur.
Als eindresultaat wil ik dus bijvoorbeeld het volgende zien (kleuren zijn niet noodzakelijk correct):
* mate          white
* outoftime     black
* draw          black
* resign        white

In [ ]:
%%file oefening_3.py
# uitkomsten per kleur

from mrjob.job import MRJob
import csv
from io import StringIO

col_victory_status = 5
col_winner = 6
col_white_rating = 9
col_black_rating = 11

class Vraag3(MRJob):

    
    def mapper(self, _, line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return

        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file))    

        yield (cols[col_victory_status], cols[col_winner])
        
    def reducer(self, word, counts):

        counts = list(counts)

        count_white = counts.count('white')
        count_black = counts.count('black')

        yield (word, 'white' if count_white >= count_black else 'black')
        

if __name__ == '__main__':
    Vraag3.run()


In [ ]:
# commando voor uitvoeren van de applicatie
!python3 oefening_3.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

## Aantal keer schaak per spelletje

Schaak tel je door een '+' teken in de moves kolom. Schaakmat door een '#' teken. Bereken de volgende statistieken van het aantal keer dat er schaak (schaakmat tel je ook mee) is per schaakspelletje:
* Minimum
* Maximum
* Gemiddelde
* Standaardafwijking

In [11]:
%%file oefening_4.py
# aantal keer shcaak
from mrjob.job import MRJob
import csv
from io import StringIO
import numpy as np

col_victory_status = 5
col_winner = 6
col_white_rating = 9
col_black_rating = 11
col_moves = 12

class Vraag4(MRJob):

    
    def mapper(self, _, line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return

        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file))    

        aantal_keer_schaak = cols[col_moves].count('+')
        aantal_keer_mat = cols[col_moves].count('#')
        
        yield ("aantal schaak", aantal_keer_mat + aantal_keer_schaak)
        
    def reducer(self, word, counts):

        counts = list(counts)

        yield("min", min(counts))
        yield("max", max(counts))
        
        yield("mean", sum(counts) / len(counts))
        yield("std", np.std(counts))

if __name__ == '__main__':
    Vraag4.run()

Writing oefening_4.py


In [12]:
# commando voor uitvoeren van de applicatie
!python3 oefening_4.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/oefening_4.root.20240319.080001.741313
uploading working dir files to hdfs:///user/root/tmp/mrjob/oefening_4.root.20240319.080001.741313/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/oefening_4.root.20240319.080001.741313/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar2313002637818764879/] [] /tmp/streamjob8416503118722809363.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032
  Connecting to Application History server at historyserver/172.20.0.9:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032

## Verificatie van de kolom met het aantal zetten

In de dataset zijn twee manieren aanwezig om het aantal zetten in een schaakspel te bekomen.
* Er is een turns kolom dat het totaal aantal beurten vast heeft
* Er is ook een moves kolom dat alle uitgevoerde zetten bevat (gescheidden door een spatie)

Kijk of deze waarden voor alle rijen overeenkomt en tel de volgende waarden:
* Totaal aantal schaakspelletjes
* Aantal waar de kolommen overeenkomen
* Aantal waar de kolommen niet overeenkomen.

Zorg ervoor dat deze drie waarden aanwezig zijn in het uiteindelijke resultaat.

In [ ]:
%%file oefening_5.py
from mrjob.job import MRJob
import csv
from io import StringIO
import numpy as np

col_turns = 4
col_victory_status = 5
col_winner = 6
col_white_rating = 9
col_black_rating = 11
col_moves = 12

class Vraag5(MRJob):

    
    def mapper(self, _, line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return

        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file)) 

        turns = int(cols[col_turns])
        moves = cols[col_moves].count(' ') + 1 # plus 1 voor de eerste zet

        yield ("aantal zetten matches", 1 if turns == moves else 0)
        
    def reducer(self, word, counts):

        counts = list(counts)

        yield ('aantal', len(counts))
        yield ('overeenkomst', sum(counts))
        yield ('niet-overeenkomst', len(counts) - sum (counts))
        

if __name__ == '__main__'

In [ ]:
# commando voor uitvoeren van de applicatie
!python3 oefening_5.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

## De verschillende openingen

Schrijf nu een map-reduce applicatie dat het aantal verschillende openingen in de dataset telt. Tel daarnaast ook hoe vaak elke opening voorkomt. Gebruik hiervoor de naam van de openingen. Tip: een set in python bevat geen duplicaten

In [ ]:
%%file oefening_6.py

from mrjob.job import MRJob
import csv
from io import StringIO
import numpy as np

col_turns = 4
col_victory_status = 5
col_winner = 6
col_white_rating = 9
col_black_rating = 11
col_moves = 12
col_openingen = 14

class Vraag6(MRJob):
    def mapper(self,_,line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return

        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file)) 
        
        yield("opening", cols[col_openingen])
        
        # aantal keer dat elke opening voorkomt
        #yield(cols[col_openingen], 1)

    def reducer(self, key, counts):
        
        counts = list(counts)
        unieke = set(counts)

        yield("aantal unieke", len(unieke))

        # manier 2 voor het aantal openingen te tellen
        for opening in unieke:
            yield(opening, counts.count(opening))
        
    
if __name__ == '__main__':
    Vraag6.run()

In [ ]:
# commando voor uitvoeren van de applicatie
!python3 oefening_6.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

## verscheidene statistieken in 1 applicatie

Maak nu een map-reduce applicatie dat de volgende zaken in 1 keer berekend:
* Het minimum, maximum en gemiddelde aantal minuten dat een schaakspel geduurd heeft (de tijden aanwezig in de dataset zijn in [Unix-format](https://en.wikipedia.org/wiki/Unix_time))
* Een histogram van het aantal zetten in een schaakspel. Plaats de grensen van de bins in de naam van de resultaat op de volgende manier: bin_{ondergrens}_{bovengrens}. Gebruik hierbij 10 bins die het bereik van 1-51 verdelen (0 zetten worden dus niet geteld).
* De openingsstrategie die het vaakst leidt tot winst voor de witte speler. Tip: collections package

In [19]:
%%file oefening_7.py

from mrjob.job import MRJob
import csv
from io import StringIO
import numpy as np
from collections import Counter

col_start = 2
col_last_move = 3
col_turns = 4
col_victory_status = 5
col_winner = 6
col_white_rating = 9
col_black_rating = 11
col_moves = 12
col_openingen = 14

class Vraag7(MRJob):

    
    def mapper(self, _, line):

        if line == "id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply":
            return

        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file)) 

        last_move = float(cols[col_last_move])
        first_move = float(cols[col_start])
        yield("aantal minuten", (last_move - first_move) / 1000 / 60)

        aantal_zetten = int(cols[col_turns])
        if aantal_zetten > 0:
            yield("aantal zetten", aantal_zetten)

        if cols[col_winner] == 'white':
            yield("opening", cols[col_openingen])

    def reducer(self, key, counts):

        counts = list(counts)

        if key == "aantal minuten":
            yield('min', min(counts))
            yield('max', max(counts))
            yield('mean', sum(counts)/len(counts))
        elif key == 'opening':
            counts = Counter(counts)
            yield('beste strategie', counts.most_common(1)[0])
        elif key == 'aantal zetten':
            bin_limits = [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51]
            hist, bin_edges = np.histogram(counts, bins=bin_limits)

            for i in range(len(bin_edges) - 1):
                yield(f"bin_{bin_edges[i]}_{bin_edges[i+1]}", int(hist[i]))

if __name__ == '__main__':
    Vraag7.run()

Overwriting oefening_7.py


In [20]:
# commando voor uitvoeren van de applicatie
!python3 oefening_7.py -r hadoop hdfs:///user/bigdata/Oef_MapReduce/games.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/oefening_7.root.20240319.082020.760613
uploading working dir files to hdfs:///user/root/tmp/mrjob/oefening_7.root.20240319.082020.760613/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/oefening_7.root.20240319.082020.760613/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar8063382338649587036/] [] /tmp/streamjob7534928226200867496.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032
  Connecting to Application History server at historyserver/172.20.0.9:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.7:8032